# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import SimpleExpSmoothing

In [2]:
# Read the dataset
cleaned_df = pd.read_csv('/Users/priyakundu/Documents/NYU Capstone WaterVue Files/Data Processing/Updated_Dataframe_WaterQual.csv')
cleaned_df

,Location,Sample Date,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
0,#1 HILLSBORO CANAL US 1,2006-02-28,12.100000,6.980000,19.400000,10.351373,0.830000,0.086000,2.400000
1,#1 HILLSBORO CANAL US 1,2006-08-31,4.430000,5.540000,15.750000,10.165844,0.981000,0.109000,1.400000
2,#1 HILLSBORO CANAL US 1,2007-02-28,2.605000,6.195000,23.200000,10.501905,0.754000,0.083500,1.650000
3,#1 HILLSBORO CANAL US 1,2007-08-31,4.890000,4.730000,31.100000,10.774781,0.777000,0.094000,2.300000
4,#1 HILLSBORO CANAL US 1,2008-02-29,5.925000,5.770000,12.500000,9.893361,1.440000,0.102000,2.050000
...,...,...,...,...,...,...,...,...,...
853,#89 NOB HILL RD POMPANO CANAL,2020-02-29,3.933333,5.703333,0.310000,6.451930,1.097133,0.014000,0.750000
854,#89 NOB HILL RD POMPANO CANAL,2020-08-31,1.570000,7.760000,0.250000,6.265301,0.910000,0.011000,0.000000
855,#89 NOB HILL RD POMPANO CANAL,2021-02-28,2.966667,5.833333,0.233333,6.153383,0.984033,0.003667,0.566667
856,#89 NOB HILL RD POMPANO CANAL,2021-08-31,5.510000,5.370000,0.270000,6.324359,1.410500,0.049000,0.775000


## Splitting

In [3]:
sorted_df = cleaned_df.sort_values(by='Sample Date', ascending=True)

In [4]:
# Split data into train and test sets
train_data = sorted_df.iloc[:-3*cleaned_df["Location"].nunique()].sort_values(by=['Location', 'Sample Date'], ascending=True)  # Use all but the last 12 months for training
test_data = sorted_df.iloc[-3*cleaned_df["Location"].nunique():].sort_values(by=['Location', 'Sample Date'], ascending=True)   # Use the last 12 months for testing

# Forecasting

In [5]:
attributes = ['Chlorophyll A', 'Dissolved Oxygen', 'Salinity', 'Specific Conductance', 'Total Nitrogen', 'Total Phosphorus', 'Turbidity']

In [6]:
final_dict = {}
for location in train_data["Location"].unique():
	att_rmse = {}
	train_df = train_data[train_data["Location"] == location].set_index("Sample Date")
	train_df.index = pd.to_datetime(train_df.index)  # Convert index to DateTimeIndex
	test_df = test_data[test_data["Location"]==location]

	for attribute in attributes:
		rmses = []
		for alpha in [_/10 for _ in range(1, 10, 1)]:
			model = SimpleExpSmoothing(train_df[attribute])
			result = model.fit(smoothing_level=alpha, optimized=False)
			
			# Forecast future values
			forecast = result.forecast(steps=3)  # Forecasting next 3 periods into the future

			mse = mean_squared_error(test_df[attribute], forecast)
			rmse = np.sqrt(mse)
			rmses.append(rmse)
		att_rmse[attribute] = rmses
	att_rmse_df = pd.DataFrame(att_rmse)

	# Initialize MinMaxScaler
	scaler = MinMaxScaler()

	# Fit and transform the selected columns
	att_rmse_df[attributes] = scaler.fit_transform(att_rmse_df[attributes].values)
	final_dict[location] = round((att_rmse_df.sum(axis=1).argmin()+1)*0.1, 1)
	
final_dict_df = pd.DataFrame({"Locations": final_dict.keys(), "Alpha": final_dict.values()})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library

In [7]:
final_dict_df

,Locations,Alpha
0,#1 HILLSBORO CANAL US 1,0.4
1,#10 MIDDLE RIVER E SUNRISE,0.3
2,#110 POMPANO CANAL AT DIXIE AN,0.3
3,#111 S. FORK MID R. @ N.E. 15,0.1
4,#12 MIDDLE RIVER NW 31ST AVE,0.6
5,#15 NEW RIVER ANDREWS AVE,0.9
6,#17 PLANTATION CANAL @ S-33,0.3
7,#19 NEW RIVER RIVER REACH,0.5
8,#22 N NEW RIVER SW 125 AVE,0.4
9,#24 DANIA CUT-OFF US-1,0.5


In [8]:
att_rmse = {attribute: [] for attribute in attributes}

for location in final_dict:
	alpha = final_dict[location]

	train_df = train_data[train_data["Location"] == location].set_index("Sample Date")
	train_df.index = pd.to_datetime(train_df.index)  # Convert index to DateTimeIndex
	test_df = test_data[test_data["Location"]==location]

	for attribute in attributes:
		model = SimpleExpSmoothing(train_df[attribute])
		result = model.fit(smoothing_level=alpha, optimized=False)
		
		# Forecast future values
		forecast = result.forecast(steps=3)  # Forecasting next 3 periods into the future

		mse = mean_squared_error(test_df[attribute], forecast)
		rmse = np.sqrt(mse)
		att_rmse[attribute].append(rmse)

final_dict_df = pd.concat([final_dict_df, pd.DataFrame(att_rmse)], axis=1)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 2QE-NOV will be used.
  self._init_dates(dates, freq)
/Library

In [9]:
final_dict_df

,Locations,Alpha,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
0,#1 HILLSBORO CANAL US 1,0.4,1.420526,0.235522,5.855080,0.936490,0.005385,0.006321,0.349862
1,#10 MIDDLE RIVER E SUNRISE,0.3,2.078647,0.212909,2.824929,0.156923,0.211986,0.039650,0.435715
2,#110 POMPANO CANAL AT DIXIE AN,0.3,2.133083,0.707459,0.017466,0.100168,0.124261,0.012880,0.721996
3,#111 S. FORK MID R. @ N.E. 15,0.1,2.848059,0.696950,7.543931,1.365365,0.214018,0.018058,0.220088
4,#12 MIDDLE RIVER NW 31ST AVE,0.6,3.959329,1.040007,0.004106,0.024587,0.083800,0.006207,0.355881
5,#15 NEW RIVER ANDREWS AVE,0.9,1.193588,0.908946,10.213668,1.111225,0.320262,0.038693,0.262770
6,#17 PLANTATION CANAL @ S-33,0.3,2.140329,2.322025,9.756381,1.533494,0.327481,0.012393,0.248203
7,#19 NEW RIVER RIVER REACH,0.5,2.420416,0.663543,4.378769,1.072554,0.188374,0.015777,0.492943
8,#22 N NEW RIVER SW 125 AVE,0.4,1.010786,1.316968,0.017451,0.046950,0.104204,0.009427,0.483733
9,#24 DANIA CUT-OFF US-1,0.5,0.872099,0.500021,7.583276,0.466405,0.219888,0.026748,0.647752


In [11]:
accuracy = 1 - (final_dict_df.iloc[:,2:]/final_dict_df.iloc[:,2:].max())

accuracy.insert(0,"Locations", final_dict_df["Locations"])
accuracy.insert(1,"Alpha", final_dict_df["Alpha"])

accuracy

,Locations,Alpha,Chlorophyll A,Dissolved Oxygen,Salinity,Specific Conductance,Total Nitrogen,Total Phosphorus,Turbidity
0,#1 HILLSBORO CANAL US 1,0.4,0.732455,0.898570,0.426741,0.389309,0.985889,0.840572,0.638950
1,#10 MIDDLE RIVER E SUNRISE,0.3,0.608503,0.908309,0.723417,0.897670,0.444506,0.000000,0.550351
2,#110 POMPANO CANAL AT DIXIE AN,0.3,0.598250,0.695327,0.998290,0.934680,0.674383,0.675148,0.254915
3,#111 S. FORK MID R. @ N.E. 15,0.1,0.463590,0.699853,0.261389,0.109638,0.439183,0.544568,0.772874
4,#12 MIDDLE RIVER NW 31ST AVE,0.6,0.254291,0.552112,0.999598,0.983967,0.780408,0.843465,0.632738
5,#15 NEW RIVER ANDREWS AVE,0.9,0.775197,0.608555,0.000000,0.275364,0.160778,0.024137,0.728827
6,#17 PLANTATION CANAL @ S-33,0.3,0.596886,0.000000,0.044772,0.000000,0.141862,0.687439,0.743859
7,#19 NEW RIVER RIVER REACH,0.5,0.544133,0.714239,0.571283,0.300581,0.506381,0.602085,0.491294
8,#22 N NEW RIVER SW 125 AVE,0.4,0.809626,0.432837,0.998291,0.969384,0.726942,0.762253,0.500798
9,#24 DANIA CUT-OFF US-1,0.5,0.835747,0.784662,0.257536,0.695855,0.423802,0.325388,0.331534


In [ ]:
accuracy.describe()